In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

m = args["num_trials"]
random_seeds = np.loadtxt("random_seeds.txt").astype("int64")
env_seeds = random_seeds[:m]
simulator_seeds = random_seeds[m:2*m]
mcts_seeds = random_seeds[2*m:]

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P, simulator_seed=0)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, env_seed, simulator_seed, mcts_seed, args):

    # random.seed(random_seeds[i_trial])
    # np.random.seed(random_seeds[i_trial])

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P, simulator_seed)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, env_seed, mcts_seed, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [3]:
args["update_method"] = "avg"
args["rollout_method"] = ""

print(f"num_trials = {args['num_trials']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")

Q_mcts_dict = defaultdict()



hparam_ucb_scale_list = np.arange(10, 100, 10)
# hparam_ucb_scale_list = np.arange(20, 64, 4)
hparam_ucb_scale_list = [32, 64, 128, 256, 512, 1024]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]


# num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
num_trajectories_list = [200, 500, 100]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    for hparam_ucb_scale in hparam_ucb_scale_list:
        # start_time = time.time()

        # print(f"hparam_ucb_scale = {hparam_ucb_scale}")
        args["hparam_ucb_scale"] = hparam_ucb_scale
        
        pool = mp.Pool()
        pool.starmap(
                run_trial, 
                [(i, Q_vit, env_seeds[i], simulator_seeds[i], mcts_seeds[i], args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        reward_error = reward_std/np.sqrt(args["num_trials"])
        if hparam_ucb_scale <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        print(f"reward = {reward_mean:0.2f} +/- {reward_error:0.2f}")

        # Q_mcts_dict[f"{hparam_ucb_scale}"] = copy.deepcopy(Q_mcts_list)

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_ucb_scale
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")
    
    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 500

-> num_trajectories = 200
reward = -56.13 +/- 2.16
reward = -64.07 +/- 2.11
reward = -80.06 +/- 1.79
reward = -75.07 +/- 1.93
reward = -80.31 +/- 1.79
reward = -78.00 +/- 1.85
max_reward_mean = -56.13
it takes 45.2519

-> num_trajectories = 500
reward = -23.82 +/- 1.75
reward = -31.94 +/- 1.97
reward = -63.79 +/- 2.11
reward = -71.17 +/- 2.01
reward = -81.66 +/- 1.74
reward = -88.06 +/- 1.48
max_reward_mean = -23.82
it takes 112.3635

-> num_trajectories = 100
reward = -71.20 +/- 2.01
reward = -77.66 +/- 1.86
reward = -78.94 +/- 1.83
reward = -79.63 +/- 1.81
reward = -78.43 +/- 1.85
reward = -83.13 +/- 1.70
max_reward_mean = -71.20
it takes 24.5217


In [4]:
print(best_param_list)

[32, 32, 32]


In [5]:
print(res_text1)
print(res_text2)

200 \\ 
 \hline 
500 \\ 
 \hline 
100 \\ 
 \hline 

200 & -56.13 (±2.16) & -64.07 (±2.11) & -80.06 (±1.79) & -75.07 (±1.93) & -80.31 (±1.79) & -78.00 (±1.85) \\ 
 \hline 
500 & -23.82 (±1.75) & -31.94 (±1.97) & -63.79 (±2.11) & -71.17 (±2.01) & -81.66 (±1.74) & -88.06 (±1.48) \\ 
 \hline 
100 & -71.20 (±2.01) & -77.66 (±1.86) & -78.94 (±1.83) & -79.63 (±1.81) & -78.43 (±1.85) & -83.13 (±1.70) \\ 
 \hline 

